In [ ]:
# install necessary libraries
!pip install --quiet fiona shapely pyproj rtree
!pip install --quiet geopandas
!pip install --quiet ipython-autotime
!pip install --quiet matplotlib
!pip install --quiet contextily

## Assessing Access to Healthcare Facilities:
<h5>The Case of Ho Municipal</h5>

In [ ]:
# import the relevant libraries
import folium
import requests
from folium import *
from geopy.geocoders import Nominatim
from folium.plugins import *
import branca
import pandas as pd
import contextily as cx
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from shapely.geometry import Point, LineString
%matplotlib inline
%load_ext autotime

time: 183 µs (started: 2023-04-01 03:55:51 +00:00)


### Font Properties

#### Font Families

In [ ]:
# To be able to customize the visual appearance of text in the code, external fonts can be downloaded and used, 
# instead of the default fonts.
# This code snippet is using the command line tool 'unzip' to extract the contents of multiple zip files 
# that are located in the "/content/drive/MyDrive/fonts/" directory. 

# Extract fonts from drive
!unzip "/content/drive/MyDrive/fonts/Metropolis.zip"

!unzip "/content/drive/MyDrive/fonts/candara.zip"


In [ ]:
# move fonts to fonts' directory
!mv Candara*.ttf /usr/share/fonts/truetype/
!mv Metropolis-*.otf /usr/share/fonts/truetype/
!fc-cache -f -v

### Data Retrieval

In [ ]:
# Define the file path for the shapefile containing community data
origin = "/content/drive/MyDrive/closest_facility/data/hm/origin/Ho_Mun_Com.shp"

# Define the file path for the shapefile containing health facility data
destination = "/content/drive/MyDrive/closest_facility/data/hm/destination/Ho_Mun_Health.shp"

# Load the community data from the shapefile into a GeoDataFrame using the read_file() function from the geopandas library
community = gpd.read_file(origin)

# Load the health facility data from the shapefile into a GeoDataFrame using the read_file() function from the geopandas library
health_facility = gpd.read_file(destination)


time: 385 µs (started: 2023-04-01 03:56:20 +00:00)


### Data Preprocessing

In [ ]:
# Convert all column names to lowercase using the map() method and the str.lower() function,
# and then assign the resulting array of column names back to the column attribute of the community GeoDataFrame.
community.columns = community.columns.map(str.lower)

time: 1.23 ms (started: 2023-04-01 03:56:43 +00:00)


### Visual data exploration

In [ ]:
# Create a figure object and a set of subplots using the subplots() function from the matplotlib library
fig, ax = plt.subplots(1, 1)

# Set the size of the figure using the set_size_inches() method
fig.set_size_inches(10, 10)

# Plot the community GeoDataFrame on the axes using the plot() method from the geopandas library
# with black markers ('o') as the point style
community.plot(ax=ax, color='black', marker='o')

# Add a basemap to the plot using the add_basemap() function from the contextily library (imported as cx)
# with the CRS (coordinate reference system) of the community GeoDataFrame and the OpenStreetMap.Mapnik source
cx.add_basemap(ax, crs=community.crs, source=cx.providers.OpenStreetMap.Mapnik)

# Hide the x-axis and y-axis labels using the xaxis.set_visible() and yaxis.set_visible() methods on the current axes object (gca)
plt.gca().xaxis.set_visible(False)
plt.gca().yaxis.set_visible(False)

# Show the plot
plt.show()

In [ ]:
# Define a string variable `api_key` that contains the API key for Open Route Service
api_key = "API_KEY_HERE"

time: 330 µs (started: 2023-04-01 03:56:53 +00:00)


In [ ]:
# Assign the community GeoDataFrame to the `origin_gdf` variable and the name of the community column to `origin_name_col`
origin_gdf = community
origin_name_col = 'community'

# Assign the health facility GeoDataFrame to the `destination_gdf` variable and the name of the facility name column to `destination_name_col`
destination_gdf = health_facility
destination_name_col = 'FacilityNa'

# These variables are used later in the code to compute the distance matrix between each origin and destination.

time: 359 µs (started: 2023-04-01 03:56:57 +00:00)


In [ ]:
# Create a list of origin points as tuples of (x, y) coordinates using the `geometry` attribute of `origin_gdf`
# and the `zip()` function to combine the x-coordinates and y-coordinates into a tuple
origins = list(zip(origin_gdf.geometry.x, origin_gdf.geometry.y))

# Create a list of destination points as tuples of (x, y) coordinates using the `geometry` attribute of `destination_gdf`
# and the `zip()` function to combine the x-coordinates and y-coordinates into a tuple
destinations = list(zip(destination_gdf.geometry.x, destination_gdf.geometry.y))

# Combine the origin and destination points into a single list of locations
locations = origins + destinations

# Create a list of origin indices that correspond to the index of each point in the `origins` list
# This will be used later to match the origin points with their corresponding rows in the distance matrix
origins_index = list(range(0, len(origins)))

# Create a list of destination indices that correspond to the index of each point in the `destinations` list
# This will be used later to match the destination points with their corresponding columns in the distance matrix
destinations_index = list(range(len(origins), len(locations)))

# These lists are used later in the code to match the origin points with their corresponding rows in the distance matrix, 
# and the destination points with their corresponding columns.

time: 2.68 ms (started: 2023-04-01 03:57:01 +00:00)


In [ ]:
# Define the request body as a dictionary with the necessary parameters for the Open Route Service API
# `locations`: a list of origin and destination points
# `destinations`: a list of destination indices that correspond to the index of each point in the `locations` list
# `sources`: a list of origin indices that correspond to the index of each point in the `locations` list
# `metrics`: a list of metrics to compute, in this case, only 'distance'
# `units`: the units to use for the computed distances, in this case, kilometers
body = {'locations': locations,
       'destinations': destinations_index,
       'sources': origins_index,
       'metrics': ['distance'],
        'units': 'km'}

# Define the request headers with the necessary parameters for the Open Route Service API
# `Accept`: the format for the response
# `Authorization`: the API key for authentication
# `Content-Type`: the format for the request body
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': api_key,
    'Content-Type': 'application/json; charset=utf-8'
}

# Send a POST request to the Open Route Service API with the defined request body and headers
response = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', 
                         json = body, headers = headers)


time: 304 ms (started: 2023-04-01 03:57:22 +00:00)


In [ ]:
# Extract the distances from the response returned by the Open Route Service API
distances = response.json()['distances']

time: 1.24 ms (started: 2023-04-01 03:57:32 +00:00)


### Post-processing

In [ ]:
# Initialize an empty list to store the distance matrix
distance_matrix = []

# Iterate over each origin in the origin GeoDataFrame
for origin_index, item in origin_gdf.iterrows():
  
  # Get the name and coordinates of the origin point
  origin_name = item[origin_name_col]
  origin_x = item.geometry.x
  origin_y = item.geometry.y
  
  # Get the distances from the origin point to all destination points
  origin_distances = distances[origin_index]
  
  # Find the minimum distance and the index of the corresponding destination point
  min_dist = min(origin_distances)
  min_index = origin_distances.index(min_dist)
  dest_index = destinations_index[min_index]
  
  # Get the name and coordinates of the corresponding destination point
  dest_x, dest_y = locations[dest_index]
  
  # Filter the destination GeoDataFrame to get the row corresponding to the destination point
  filtered =  destination_gdf[(destination_gdf.geometry.x == dest_x) & (destination_gdf.geometry.y == dest_y)]
  dest_row = filtered.iloc[0]
  
  # Get the name of the destination point
  dest_name = dest_row[destination_name_col]
  
  # Append the origin name, origin coordinates, destination name, destination coordinates, and distance to the distance matrix
  distance_matrix.append([origin_name, origin_y, origin_x, dest_name, dest_y, dest_x, min_dist])


time: 176 ms (started: 2023-04-01 03:57:40 +00:00)


In [ ]:
# Create a pandas DataFrame called matrix_df from the list distance_matrix with specified column names. 
# The columns are as follows:
# - origin_name: name of the community
# - origin_y: y-coordinate of the community
# - origin_x: x-coordinate of the community
# - destination_name: name of the nearest health facility
# - dest_y: y-coordinate of the nearest health facility
# - dest_x: x-coordinate of the nearest health facility
# - min_dist: distance between the community and its nearest health facility
matrix_df = pd.DataFrame(distance_matrix,
                          columns = ['origin_name', 'origin_y', 'origin_x', 'destination_name', 'dest_y', 'dest_x', 'min_dist']
                          )

time: 1.38 ms (started: 2023-04-01 03:57:53 +00:00)


In [ ]:
# Define a function called lineString that takes a row from the matrix_df DataFrame and creates a LineString object from the 
# origin and destination points. The function returns the LineString object.
def lineString(row):
    origin_point = Point(row['origin_x'], row['origin_y']) # create a Point object from the origin_x and origin_y values
    dest_point = Point(row['dest_x'], row['dest_y']) # create a Point object from the dest_x and dest_y values
    return LineString([origin_point, dest_point]) # create a LineString object from the origin and destination points

# Apply the lineString function to each row in the matrix_df DataFrame and create a GeoDataFrame called matrix_gdf.
# The geometry of each row is the LineString object created by the lineString function.
# The crs is set to EPSG:4326, which is the WGS 84 coordinate reference system.
geom = matrix_df.apply(lineString, axis = 1)
matrix_gdf = gpd.GeoDataFrame(
                                matrix_df, 
                                geometry = geom,
                                crs = 'EPSG:4326'
                             )

time: 26.2 ms (started: 2023-04-01 03:58:20 +00:00)


### Displaying Each Community and its Associated Nearest Health Facility

In [ ]:
# Set font properties

# metropolis-thin
metro_thin = "/usr/share/fonts/truetype/Metropolis-Thin.otf"
metrofontprop = fm.FontProperties(fname = metro_thin, size = 15.5)


# autho - metropolis
metro_thin = "/usr/share/fonts/truetype/Metropolis-Thin.otf"
metrofontprop_i = fm.FontProperties(fname = metro_thin, size = 12, weight = 'bold')

# candara
candara = "/usr/share/fonts/truetype/Candara.ttf"
candara_prop = fm.FontProperties(fname = candara, size = 15)

time: 549 µs (started: 2023-04-01 03:58:30 +00:00)


In [ ]:
# Create a figure and axis object
fig, ax = plt.subplots(1, 1, figsize = (12, 8))

# Plot the origin, destination, and distance matrix GeoDataFrames
origin_gdf.plot(ax = ax, color = 'blue', alpha = 1, marker = 'o')
matrix_gdf.plot(ax = ax, color = 'black', linestyle = '--', linewidth = 0.75)
destination_gdf.plot(ax = ax, color = 'red', alpha = 1, marker = '+')

# Add a basemap to the plot
cx.add_basemap(
    ax = ax,
    crs = health_facility.crs,
    source = cx.providers.OpenStreetMap.Mapnik
              )

# Create legend elements
legend_elements = [
                    plt.plot([], [], color = 'red', alpha = 1, marker = '+', label = 'Health Center', ls = '')[0],
                    plt.plot([], [], color = 'blue', alpha = 1, marker = 'o', label = 'Community', ls = '')[0],
                    plt.plot([], [], color = 'black', alpha = 1, marker = '_', label = 'Distance Matrix', ls = '')[0]
                  ]

# Add legend to the plot
ax.legend(
            handles = legend_elements,
            loc = 'lower right', prop = metrofontprop_i
         )

# Retrieve the Text objects representing the legend labels
legend_labels = ax.legend_.get_texts()

# Set the color of the 'Community' legend label to red
legend_labels[0].set_color('red')

# Set the color of the 'Health Center' legend label to blue
legend_labels[1].set_color('blue')

# Set the color of the 'Distance Matrix' legend label to black
legend_labels[2].set_color('black')

# Remove x and y axis ticks
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

# Add a title to the plot
title = ax.set_title("""
            Nearest Health Facilities  
         Among Communities in Ho Municipal
      """, fontproperties = metrofontprop, color = "black", weight = 'bold'
      )

# Save the plot to an output folder
output_folder = "/content/drive/MyDrive/closest_facility/output"
output = "/content/sample_data/output"
plt.savefig(output_folder+'/closestFacs_land.jpg',  dpi = 300, bbox_inches = 'tight', orientation = 'landscape')

# Display the plot
plt.show()


In [ ]:
# Compute the descriptive statistics of the 'min_dist' column in the matrix_gdf GeoDataFrame
matrix_gdf.min_dist.describe()

count    96.000000
mean      2.885208
std       2.431121
min       0.020000
25%       0.950000
50%       2.080000
75%       4.137500
max      11.220000
Name: min_dist, dtype: float64

time: 20.9 ms (started: 2023-04-01 04:01:08 +00:00)


In [ ]:
# Get the row index of the origin with the highest min_dist
max_dist_idx = matrix_gdf['min_dist'].idxmax()

# Get the name of the origin with the highest min_dist
origin_name_with_max_dist = matrix_gdf.loc[max_dist_idx, 'origin_name']
origin_y_with_max_dist = matrix_gdf.loc[max_dist_idx, 'origin_y']
origin_x_with_max_dist = matrix_gdf.loc[max_dist_idx, 'origin_x']

dest_name_with_max_dist = matrix_gdf.loc[max_dist_idx, 'destination_name']
dest_y_with_max_dist = matrix_gdf.loc[max_dist_idx, 'dest_y']
dest_x_with_max_dist = matrix_gdf.loc[max_dist_idx, 'dest_x']

# print the name of the origin, and its coordinates as well as the name of the destination and its coordinates, both with the highest min_dist
print(origin_name_with_max_dist, origin_y_with_max_dist, origin_x_with_max_dist)
print(dest_name_with_max_dist, dest_y_with_max_dist, dest_x_with_max_dist)

Sokode Ando 6.572730000000001 0.36731000000000075
Akpe Na Mawu Clinic 6.59496 0.45248000000000005
time: 1.42 ms (started: 2023-04-01 04:20:43 +00:00)


###Interactive web map for displaying communities and health facilities as well as the distance between them

In [ ]:
# Define a TileLayer for Google Street Map
google_tiles = TileLayer(
    name = "Google Street Map",
    # Use Google Maps API to fetch map tiles for a given x, y and z zoom level
    # Replace API_KEY with your google maps API KEY
    tiles = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}&key=<API_KEY>",
    # Add attribution to Google Maps
    attr = '<a href="https://www.google.com/maps/">Google Maps</a>'
)

# Create a Folium map centered on Ho, Ghana
mp = folium.Map(
    location = [6.6126598, 0.4688932], 
    width=800, 
    height=500, 
    # Use Google Street Map as the basemap
    tiles = google_tiles, 
    # Add a scale bar to the map
    control_scale = True, 
    # Set the initial zoom level
    zoom_start = 15
)

# Create a GeoJson layer for the origin points (i.e., communities)
origin_json = GeoJson(
    origin_gdf, 
    # Add a tooltip to the GeoJson layer
    tooltip = folium.GeoJsonTooltip(
        # Specify which fields to include in the tooltip
        fields = ["community", "region", "district"], 
        # Rename the fields to more user-friendly names
        aliases = ["Community", 'Region', 'District'],
        # Customize the style of the tooltip
        style = (
            "background-color: #AD1457; "
            "color: #FFFFFF; "
            "font-family: verdana; "
            "font-size: 12px; "
            "padding: 10px"
        )
    ), 
    # Hide the GeoJson layer by default
    show = False
)

# Create a GeoJson layer for the destination points (i.e., healthcare facilities)
destination_json = GeoJson(
    destination_gdf,
    # Add a tooltip to the GeoJson layer
    tooltip = folium.GeoJsonTooltip(
        # Specify which fields to include in the tooltip
        fields = ["Region", "District", "FacilityNa", "Type", "Town", "Ownership"],
        # Rename the fields to more user-friendly names
        aliases = ["Region", "District", "FacilityNa", "Type", "Town", "Ownership"],
        # Customize the style of the tooltip
        style = (
            "background-color: #AD1457; "
            "color: #FFFFFF; "
            "font-family: verdana; "
            "font-size: 12px; "
            "padding: 10px"
        )
    ), 
    # Hide the GeoJson layer by default
    show = False
)

# Define a style function for the matrix layer
def my_style(feature):
    return {
        'color': 'black',
        'weight': 2,
    }

# Create a GeoJson layer for the matrix (i.e., the lines connecting each origin to its nearest destination)
matrix_json = folium.GeoJson(
    matrix_gdf,
    name = "Distance",
    # Add a tooltip to the GeoJson layer
    tooltip = folium.GeoJsonTooltip(
        # Specify which fields to include in the tooltip
        fields = ["origin_name", "destination_name", "min_dist"],
        # Rename the fields to more user-friendly names
        aliases = ["Origin Name:", "Destination Name:", "Minimum Distance (km):"],
        # Customize the style of the tooltip
        style = "background-color: #F0EFEF; color: #000000; font-family: Arial; font-size: 12px; padding: 10px;"
    ),
    # Apply the style function to the matrix layer
    style_function = my_style
).add_to(mp)

# Create a FeatureGroup for the origin points (i.e., communities)
comm_fg = folium.FeatureGroup('Community')

# Create a FeatureGroup for the destination points (i.e., healthcare facilities)
health_fg = folium.FeatureGroup('Health Facility')


# Iterate through each feature in the origin_json data
for feature in origin_json.data['features']:
    
    # Check if the feature is a point
    if feature['geometry']['type'] == 'Point':
        
        # Create a string variable containing HTML code with information about the community
        elements = """
                <body style='background-color:#0d2b3e'>
                  <div  style='text-align:center; padding: 2px 2px'>
                    <h5>
                      <p style='color:white;  font-family: Tahoma; font-size: 12px'>Community: {}</p>
                      <p style='color:white;  font-family: Candara; font-size: 12px'>District: {}</p>  
                      <p style='color:white;  font-family: Candara; font-size: 12px'>Region: {}</p> 
                    </h5>
                  </div>
                </body>""".format(
            feature['properties']['community'], # Retrieve the community name from the feature's properties
            feature['properties']['district'], # Retrieve the district name from the feature's properties
            feature['properties']['region']) # Retrieve the region name from the feature's properties

        # Use the HTML formatted string to set the contents of the popup object for the feature
        iframe = IFrame(elements)
        

        # Create a popup object using the iframe object created earlier
        # Set the maximum and minimum widths of the popup window
        popup = Popup(iframe, max_width = 5, min_width = 200) 
        Marker(location=list(reversed(feature['geometry']['coordinates'])), 
               icon=Icon(color="blue", prefix='fa', icon="fa-home", icon_color="white"),
               popup=popup).add_to(comm_fg)


# Iterate through each feature in the destination_json data
for feature in destination_json.data['features']:
  # Check if the feature is a point
  if feature['geometry']['type'] == 'Point':
    # Create a string variable containing HTML code with information about the facility
    elements = """
                  <body style='background-color:#0d2b3e'>
                    <div style='text-align:center; padding: 3px 3px'>
                      <h5>
                        <p style='color:white;  font-family: Tahoma; font-size: 12px'>Facility Name: {}</p>
                        <p style='color:white;  font-family: Candara; font-size: 12px'>Type: {}</p> 
                        <p style='color:white;  font-family: Candara; font-size: 12px'>Locality: {}</p> 
                        <p style='color:white;  font-family: Candara; font-size: 12px'>Ownership: {}</p> 
                      </h5>
                    </div>
                  </body>
                """.format(
        feature['properties']['FacilityNa'], # Retrieve the facility name from the feature's properties
        feature['properties']['Type'], # Retrieve the facility type from the feature's properties
        feature['properties']['Town'], # Retrieve the locality from the feature's properties
        feature['properties']['Ownership'] # Retrieve the ownership from the feature's properties
        )



    # Use the HTML formatted string to set the contents of the popup object for the feature
    iframe = IFrame(elements)
    # Create a popup object using the iframe object created earlier
    # Set the maximum and minimum widths of the popup window
    # Set parse_html parameter to True to enable HTML parsing
    popup = Popup(iframe, max_width = 5, min_width = 250, parse_html = True)

    # Add a marker for the health facility feature to the health facility feature group
    # Set the location of the marker as the coordinates of the health facility feature
    # Set the icon of the marker as a red square
    # Set the popup of the marker as the popup object created earlier
    Marker(location =  list(reversed(feature['geometry']['coordinates'])),
                  icon =  Icon(color = "red", prefix = 'fa', icon = 'fa-square-h', icon_color = "white"),
                  popup = popup).add_to(health_fg)

# Add the comm_fg feature group to the mp map object
comm_fg.add_to(mp)

# Add the health_fg feature group to the mp map object
health_fg.add_to(mp)

# Created tile layers to be added to the map

TileLayer('Open Street Map').add_to(mp)
TileLayer('Stamen Terrain').add_to(mp)
TileLayer('CartoDB Dark_Matter').add_to(mp)   
TileLayer(
    tiles ='http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr ='Google',
    name ='Google Imagery'
).add_to(mp)

# Set control panel for the map
LayerControl().add_to(mp)


# Set fullscreen option
fullscreen = Fullscreen(position="top right")

# Add insert map
minimap = MiniMap(toggle_display=True, minimized = True)
mp.add_child(minimap)
mp.add_child(fullscreen)

# Save map
output = "/content/sample_data/output"
mp.save(output+"/closest_fac.html")

# Display map
mp

### Actual Routes Between Origins and Destinations

In [ ]:
# Define the start and end locations
sokode_ando = (6.572730000000001, 0.36731000000000075)
akpe_na_mawu_clinic = (6.59496, 0.45248000000000005)


# Define the OpenRouteService API key
api_key = "API KEY HERE"

# Define the API request body
body = {
    'api_key': api_key,
    'start': '{},{}'.format(sokode_ando[1], sokode_ando[0]),
    'end': '{},{}'.format(akpe_na_mawu_clinic[1], akpe_na_mawu_clinic[0])
}

# Send the API request
res = requests.get("https://api.openrouteservice.org/v2/directions/driving-car", params=body)

distance = res.json()

summary = distance['features'][0]['properties']['summary']

distance = round(summary['distance']/1000, 1)

dir = res.json()['features'][0]['geometry']['coordinates']
points = [(i[1], i[0]) for i in dir]

mp = folium.Map(location = [6.5740764, 0.4133276], width = 600, height = 450, tiles = google_tiles, control_scale = True, zoom_start = 13)

# Define custom names for the markers
start_name = 'Sokode Ando'
end_name = 'Akpe na Mawu Clinic'

for point, name in zip([points[0], points[-1]], [start_name, end_name]):
  if name == start_name:
    icon = 'fa-home'
    color = 'blue'
  else:
    icon = 'fa-clinic-medical'
    color = 'red'
  folium.Marker(point, icon = Icon(color = color, prefix = 'fa', icon = icon, icon_color = "white"),
                popup = name).add_to(mp)
  folium.PolyLine(points, weight = 5, opacity = 1, color = 'red',
                  tooltip = f'Distance between {start_name} and {end_name}: {distance} km', 
                  fontproperties = metrofontprop
                  ).add_to(mp)
# Save output
mp.save(output+"/closest_fac_dir.html")
mp